In [495]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
import json
import pandas as pd
import unicodedata
import html
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.word2vec import Word2Vec
import numpy as np
import gensim
from gensim.models.doc2vec import LabeledSentence
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense
import nltk
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [251]:
#Variables that contains the user credentials to access Twitter API 
access_token = "385992900-53Czv6OdaILuWQr06dSNDwsGsaRaMFMI7lylFTlP"
access_token_secret = "iCHOm9DmDmqKqXxKAeHePLkveyXr9JsdyvJ0wJdcoV7CY"
consumer_key = "eimP3ftxqtcWny10KW263Rw0I"
consumer_secret = "xCYPCfS2YWKb2HwP37eb3RHY4eVCZHQzEB2zYMVDEoqQhlSWIt"

In [307]:
n_dim = 200
lemmatizer = WordNetLemmatizer()
tokenizer = TweetTokenizer()
tqdm.pandas(desc="progress-bar")
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
# regex_str = [
#     emoticons_str,
#     r'<[^>]+>', # HTML tags
#     r'(?:@[\w_]+)', # @-mentions
#     r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
#     r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
#     r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
#     r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
#     r'(?:[\w_]+)', # other words
#     r'(?:\S)' # anything else
# ]

regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)


In [253]:
tweets_stream_data = []

In [565]:
tweets_full_text = []

In [255]:
#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):

    def on_data(self, data):
        tweets_stream_data.append(data)
        return True

    def on_error(self, status):
        print(status)


if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l, count = 10)

    #This line filter Twitter Streams to capture data by the keywords: 'python', 'javascript', 'ruby'
    stream.filter(track=['Narendra Modi', 'BJP'], languages = ["en"])

KeyboardInterrupt: 

In [566]:
for line in tweets_stream_data:
    try:
        tweet = json.loads(line)
        extended_tweet = tweet['retweeted_status']['extended_tweet']['full_text']
        tweets_full_text.append(html.unescape(extended_tweet).replace('\n', ' '))
    except:
        continue

In [568]:
def tokenize(tweet):
    try:
        tokens = nltk.word_tokenize(tweet.lower())
        tokens = list(filter(lambda t: not t.startswith('@'), tokens))
        tokens = list(filter(lambda t: not t.startswith('#'), tokens))
        tokens = list(filter(lambda t: not t.startswith('http'), tokens))
#         tokens = list(filter(token if emoticon_re.search(token) else token.lower() for token in tokens))
#         tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
        return tokens
    except:
        return 'NC'

In [606]:
def postprocess(data, n=1000000):
    data = data.head(n)
    data['tweet'] = data['tweet'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tweet != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

In [607]:
df_train = pd.read_csv('data/train.csv', encoding = 'Latin-1', header=None, nrows=1000000)

In [608]:
df_test = pd.DataFrame()
df_test['tweet'] = tweets_full_text
df_test.to_csv('data/test.csv', index=False)

In [609]:
test_data = postprocess(df_test)






progress-bar:   0%|          | 0/2413 [00:00<?, ?it/s]




progress-bar:   9%|▉         | 214/2413 [00:00<00:01, 2138.37it/s]




progress-bar:  18%|█▊        | 424/2413 [00:00<00:00, 2125.81it/s]




progress-bar:  26%|██▌       | 616/2413 [00:00<00:00, 2055.33it/s]




progress-bar:  33%|███▎      | 806/2413 [00:00<00:00, 2005.67it/s]




progress-bar:  41%|████▏     | 1001/2413 [00:00<00:00, 1987.60it/s]




progress-bar:  51%|█████     | 1219/2413 [00:00<00:00, 2040.10it/s]




progress-bar:  60%|█████▉    | 1439/2413 [00:00<00:00, 2083.56it/s]




progress-bar:  68%|██████▊   | 1649/2413 [00:00<00:00, 2087.52it/s]




progress-bar:  77%|███████▋  | 1848/2413 [00:00<00:00, 2053.93it/s]




progress-bar:  85%|████████▍ | 2046/2413 [00:01<00:00, 2008.73it/s]




progress-bar:  93%|█████████▎| 2242/2413 [00:01<00:00, 1954.87it/s]




progress-bar: 100%|██████████| 2413/2413 [00:01<00:00, 1982.22it/s]

In [610]:
df_new_train = pd.DataFrame(data = df_train[5])
df_new_train = df_new_train.rename(columns={5: "tweet"})

In [611]:
train_data = postprocess(df_new_train)






progress-bar:   0%|          | 0/1000000 [00:00<?, ?it/s]




progress-bar:   0%|          | 361/1000000 [00:00<04:37, 3599.14it/s]




progress-bar:   0%|          | 807/1000000 [00:00<04:21, 3819.21it/s]




progress-bar:   0%|          | 1288/1000000 [00:00<04:05, 4070.15it/s]




progress-bar:   0%|          | 1720/1000000 [00:00<04:01, 4140.28it/s]




progress-bar:   0%|          | 2181/1000000 [00:00<03:53, 4268.42it/s]




progress-bar:   0%|          | 2642/1000000 [00:00<03:48, 4363.04it/s]




progress-bar:   0%|          | 3104/1000000 [00:00<03:44, 4436.20it/s]




progress-bar:   0%|          | 3564/1000000 [00:00<03:42, 4482.41it/s]




progress-bar:   0%|          | 4017/1000000 [00:00<03:41, 4492.46it/s]




progress-bar:   0%|          | 4454/1000000 [00:01<03:44, 4444.27it/s]




progress-bar:   0%|          | 4958/1000000 [00:01<03:35, 4606.74it/s]




progress-bar:   1%|          | 5419/1000000 [00:01<03:36, 4603.27it/s]




progress-bar:   1%|          | 5876

progress-bar:   4%|▍         | 41126/1000000 [00:09<03:31, 4543.60it/s]




progress-bar:   4%|▍         | 41590/1000000 [00:09<03:29, 4571.61it/s]




progress-bar:   4%|▍         | 42067/1000000 [00:09<03:26, 4627.96it/s]




progress-bar:   4%|▍         | 42531/1000000 [00:09<03:31, 4520.74it/s]




progress-bar:   4%|▍         | 42985/1000000 [00:09<03:39, 4355.14it/s]




progress-bar:   4%|▍         | 43463/1000000 [00:09<03:33, 4471.33it/s]




progress-bar:   4%|▍         | 43913/1000000 [00:09<03:37, 4390.99it/s]




progress-bar:   4%|▍         | 44355/1000000 [00:09<03:37, 4397.53it/s]




progress-bar:   4%|▍         | 44797/1000000 [00:09<03:41, 4308.16it/s]




progress-bar:   5%|▍         | 45233/1000000 [00:10<03:40, 4323.42it/s]




progress-bar:   5%|▍         | 45667/1000000 [00:10<03:44, 4257.84it/s]




progress-bar:   5%|▍         | 46095/1000000 [00:10<03:43, 4263.43it/s]




progress-bar:   5%|▍         | 46574/1000000 [00:10<03:36, 4408.20it/s]




progress-bar

progress-bar:   8%|▊         | 81891/1000000 [00:18<03:45, 4079.38it/s]




progress-bar:   8%|▊         | 82312/1000000 [00:18<03:42, 4116.22it/s]




progress-bar:   8%|▊         | 82725/1000000 [00:18<03:44, 4078.22it/s]




progress-bar:   8%|▊         | 83156/1000000 [00:18<03:41, 4143.74it/s]




progress-bar:   8%|▊         | 83583/1000000 [00:18<03:39, 4180.24it/s]




progress-bar:   8%|▊         | 84005/1000000 [00:18<03:38, 4190.72it/s]




progress-bar:   8%|▊         | 84440/1000000 [00:18<03:36, 4229.14it/s]




progress-bar:   8%|▊         | 84877/1000000 [00:19<03:34, 4269.06it/s]




progress-bar:   9%|▊         | 85305/1000000 [00:19<03:35, 4246.43it/s]




progress-bar:   9%|▊         | 85760/1000000 [00:19<03:31, 4328.61it/s]




progress-bar:   9%|▊         | 86201/1000000 [00:19<03:30, 4349.61it/s]




progress-bar:   9%|▊         | 86637/1000000 [00:19<03:37, 4207.52it/s]




progress-bar:   9%|▊         | 87066/1000000 [00:19<03:35, 4231.13it/s]




progress-bar

progress-bar:  12%|█▏        | 122218/1000000 [00:27<03:14, 4517.27it/s]




progress-bar:  12%|█▏        | 122672/1000000 [00:27<03:16, 4475.47it/s]




progress-bar:  12%|█▏        | 123122/1000000 [00:27<03:18, 4407.42it/s]




progress-bar:  12%|█▏        | 123565/1000000 [00:27<03:20, 4377.50it/s]




progress-bar:  12%|█▏        | 124007/1000000 [00:27<03:19, 4385.90it/s]




progress-bar:  12%|█▏        | 124447/1000000 [00:27<03:19, 4389.86it/s]




progress-bar:  12%|█▏        | 124898/1000000 [00:28<03:17, 4421.49it/s]




progress-bar:  13%|█▎        | 125341/1000000 [00:28<03:19, 4384.27it/s]




progress-bar:  13%|█▎        | 125792/1000000 [00:28<03:17, 4419.79it/s]




progress-bar:  13%|█▎        | 126235/1000000 [00:28<03:18, 4408.62it/s]




progress-bar:  13%|█▎        | 126685/1000000 [00:28<03:17, 4430.74it/s]




progress-bar:  13%|█▎        | 127143/1000000 [00:28<03:15, 4472.11it/s]




progress-bar:  13%|█▎        | 127600/1000000 [00:28<03:13, 4499.06it/s]





progress-bar:  16%|█▌        | 161608/1000000 [00:36<03:01, 4610.47it/s]




progress-bar:  16%|█▌        | 162070/1000000 [00:36<03:02, 4598.51it/s]




progress-bar:  16%|█▋        | 162545/1000000 [00:36<03:00, 4642.72it/s]




progress-bar:  16%|█▋        | 163014/1000000 [00:36<02:59, 4656.77it/s]




progress-bar:  16%|█▋        | 163480/1000000 [00:36<03:00, 4624.07it/s]




progress-bar:  16%|█▋        | 163961/1000000 [00:37<02:58, 4677.14it/s]




progress-bar:  16%|█▋        | 164446/1000000 [00:37<02:56, 4726.78it/s]




progress-bar:  16%|█▋        | 164920/1000000 [00:37<03:05, 4491.17it/s]




progress-bar:  17%|█▋        | 165372/1000000 [00:37<03:07, 4459.78it/s]




progress-bar:  17%|█▋        | 165820/1000000 [00:37<03:07, 4460.62it/s]




progress-bar:  17%|█▋        | 166268/1000000 [00:37<03:09, 4411.02it/s]




progress-bar:  17%|█▋        | 166717/1000000 [00:37<03:07, 4432.78it/s]




progress-bar:  17%|█▋        | 167198/1000000 [00:37<03:03, 4536.44it/s]





...

progress-bar:  81%|████████▏ | 813524/1000000 [03:03<00:44, 4146.89it/s]




progress-bar:  81%|████████▏ | 813940/1000000 [03:03<00:45, 4122.22it/s]




progress-bar:  81%|████████▏ | 814380/1000000 [03:03<00:44, 4200.86it/s]




progress-bar:  81%|████████▏ | 814806/1000000 [03:03<00:43, 4217.94it/s]




progress-bar:  82%|████████▏ | 815264/1000000 [03:03<00:42, 4318.47it/s]




progress-bar:  82%|████████▏ | 815704/1000000 [03:03<00:42, 4341.67it/s]




progress-bar:  82%|████████▏ | 816143/1000000 [03:03<00:42, 4355.20it/s]




progress-bar:  82%|████████▏ | 816580/1000000 [03:04<00:42, 4342.67it/s]




progress-bar:  82%|████████▏ | 817019/1000000 [03:04<00:42, 4356.19it/s]




progress-bar:  82%|████████▏ | 817482/1000000 [03:04<00:41, 4434.16it/s]




progress-bar:  82%|████████▏ | 817926/1000000 [03:04<00:41, 4411.80it/s]




progress-bar:  82%|████████▏ | 818368/1000000 [03:04<00:41, 4373.96it/s]




progress-bar:  82%|████████▏ | 818806/1000000 [03:04<00:41, 4361.64it/s]





progress-bar:  85%|████████▌ | 852742/1000000 [03:12<00:34, 4250.62it/s]




progress-bar:  85%|████████▌ | 853168/1000000 [03:12<00:35, 4189.23it/s]




progress-bar:  85%|████████▌ | 853611/1000000 [03:12<00:34, 4257.16it/s]




progress-bar:  85%|████████▌ | 854038/1000000 [03:12<00:34, 4239.92it/s]




progress-bar:  85%|████████▌ | 854492/1000000 [03:12<00:33, 4325.31it/s]




progress-bar:  85%|████████▌ | 854926/1000000 [03:12<00:34, 4212.20it/s]




progress-bar:  86%|████████▌ | 855377/1000000 [03:13<00:33, 4296.53it/s]




progress-bar:  86%|████████▌ | 855808/1000000 [03:13<00:33, 4273.06it/s]




progress-bar:  86%|████████▌ | 856237/1000000 [03:13<00:33, 4247.57it/s]




progress-bar:  86%|████████▌ | 856663/1000000 [03:13<00:34, 4172.70it/s]




progress-bar:  86%|████████▌ | 857101/1000000 [03:13<00:33, 4231.82it/s]




progress-bar:  86%|████████▌ | 857525/1000000 [03:13<00:33, 4199.45it/s]




progress-bar:  86%|████████▌ | 857946/1000000 [03:13<00:33, 4187.54it/s]





progress-bar:  89%|████████▉ | 891593/1000000 [03:21<00:25, 4239.57it/s]




progress-bar:  89%|████████▉ | 892018/1000000 [03:21<00:25, 4233.29it/s]




progress-bar:  89%|████████▉ | 892477/1000000 [03:21<00:24, 4332.45it/s]




progress-bar:  89%|████████▉ | 892911/1000000 [03:21<00:24, 4324.73it/s]




progress-bar:  89%|████████▉ | 893344/1000000 [03:21<00:25, 4188.95it/s]




progress-bar:  89%|████████▉ | 893800/1000000 [03:21<00:24, 4292.04it/s]




progress-bar:  89%|████████▉ | 894231/1000000 [03:22<00:24, 4275.96it/s]




progress-bar:  89%|████████▉ | 894660/1000000 [03:22<00:24, 4237.28it/s]




progress-bar:  90%|████████▉ | 895112/1000000 [03:22<00:24, 4315.43it/s]




progress-bar:  90%|████████▉ | 895578/1000000 [03:22<00:23, 4411.30it/s]




progress-bar:  90%|████████▉ | 896021/1000000 [03:22<00:23, 4356.28it/s]




progress-bar:  90%|████████▉ | 896458/1000000 [03:22<00:23, 4328.51it/s]




progress-bar:  90%|████████▉ | 896906/1000000 [03:22<00:23, 4371.82it/s]





progress-bar:  93%|█████████▎| 930845/1000000 [03:30<00:15, 4328.15it/s]




progress-bar:  93%|█████████▎| 931279/1000000 [03:30<00:16, 4217.65it/s]




progress-bar:  93%|█████████▎| 931707/1000000 [03:30<00:16, 4236.12it/s]




progress-bar:  93%|█████████▎| 932132/1000000 [03:30<00:16, 4205.90it/s]




progress-bar:  93%|█████████▎| 932554/1000000 [03:30<00:16, 4096.05it/s]




progress-bar:  93%|█████████▎| 932965/1000000 [03:31<00:16, 4039.66it/s]




progress-bar:  93%|█████████▎| 933370/1000000 [03:31<00:16, 4040.93it/s]




progress-bar:  93%|█████████▎| 933802/1000000 [03:31<00:16, 4120.40it/s]




progress-bar:  93%|█████████▎| 934221/1000000 [03:31<00:15, 4138.87it/s]




progress-bar:  93%|█████████▎| 934636/1000000 [03:31<00:15, 4111.92it/s]




progress-bar:  94%|█████████▎| 935071/1000000 [03:31<00:15, 4178.45it/s]




progress-bar:  94%|█████████▎| 935490/1000000 [03:31<00:15, 4079.92it/s]




progress-bar:  94%|█████████▎| 935921/1000000 [03:31<00:15, 4145.35it/s]





progress-bar:  97%|█████████▋| 969700/1000000 [03:39<00:06, 4356.14it/s]




progress-bar:  97%|█████████▋| 970136/1000000 [03:39<00:07, 4243.22it/s]




progress-bar:  97%|█████████▋| 970597/1000000 [03:39<00:06, 4345.56it/s]




progress-bar:  97%|█████████▋| 971033/1000000 [03:39<00:06, 4345.70it/s]




progress-bar:  97%|█████████▋| 971473/1000000 [03:40<00:06, 4361.50it/s]




progress-bar:  97%|█████████▋| 971910/1000000 [03:40<00:06, 4348.58it/s]




progress-bar:  97%|█████████▋| 972346/1000000 [03:40<00:06, 4303.80it/s]




progress-bar:  97%|█████████▋| 972777/1000000 [03:40<00:06, 4239.80it/s]




progress-bar:  97%|█████████▋| 973202/1000000 [03:40<00:06, 4188.18it/s]




progress-bar:  97%|█████████▋| 973627/1000000 [03:40<00:06, 4206.21it/s]




progress-bar:  97%|█████████▋| 974053/1000000 [03:40<00:06, 4221.70it/s]




progress-bar:  97%|█████████▋| 974476/1000000 [03:40<00:06, 4111.19it/s]




progress-bar:  97%|█████████▋| 974888/1000000 [03:40<00:06, 4067.67it/s]





In [612]:
X_train = train_data['tweet']
X_test = test_data['tweet']
y_train = df_train[0].map( {4:1, 0:0} )

In [613]:
print(X_train.shape, X_test.shape, y_train.shape)

(1000000,) (2413,) (1000000,)


In [614]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

In [615]:
X_train = labelizeTweets(X_train, 'TRAIN')
X_test = labelizeTweets(X_test, 'TEST')






0it [00:00, ?it/s]/Users/synerzip/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """





9660it [00:00, 95892.27it/s]




32723it [00:00, 162850.47it/s]




55593it [00:00, 184737.89it/s]




78401it [00:00, 195490.34it/s]




103169it [00:00, 205965.19it/s]




127899it [00:00, 212800.75it/s]




152347it [00:00, 217313.18it/s]




175235it [00:00, 218764.33it/s]




199571it [00:00, 221453.50it/s]




222377it [00:01, 222104.71it/s]




245516it [00:01, 222966.93it/s]




269638it [00:01, 224484.11it/s]




293884it [00:01, 225844.69it/s]




317653it [00:01, 226709.28it/s]




341379it [00:01, 227402.42it/s]




366021it [00:01, 228605.66it/s]




390379it [00:01, 229476.36it/s]




414444it [00:01, 229806.22it/s]




438348it [00:01, 230088.74it/s]




462137it [00:03, 142775.45it/s]




486596it [00:03, 145827.80it/s]




510781it [00:03, 14

In [616]:
tweet_w2v = Word2Vec(size=n_dim, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm(X_train)])
tweet_w2v.train([x.words for x in tqdm(X_train)],
                total_examples = tweet_w2v.corpus_count, epochs = tweet_w2v.epochs)






  0%|          | 0/1000000 [00:00<?, ?it/s]




 21%|██        | 211471/1000000 [00:00<00:00, 2099554.66it/s]




 41%|████      | 412495/1000000 [00:00<00:00, 2052869.21it/s]




 62%|██████▏   | 618654/1000000 [00:00<00:00, 2056251.59it/s]




 86%|████████▌ | 855901/1000000 [00:00<00:00, 2135853.86it/s]




100%|██████████| 1000000/1000000 [00:00<00:00, 2183045.00it/s]




  0%|          | 0/1000000 [00:00<?, ?it/s]




 21%|██        | 207560/1000000 [00:00<00:00, 2060315.28it/s]




 42%|████▏     | 415427/1000000 [00:00<00:00, 2068963.34it/s]




 63%|██████▎   | 632877/1000000 [00:00<00:00, 2103531.96it/s]




 86%|████████▌ | 855217/1000000 [00:00<00:00, 2133311.81it/s]




100%|██████████| 1000000/1000000 [00:00<00:00, 2190820.53it/s]

(54949432, 79799610)

In [617]:
tweet_w2v.most_similar('good')

/Users/synerzip/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('goood', 0.7213504910469055),
 ('great', 0.6987514495849609),
 ('gd', 0.6336985230445862),
 ('rough', 0.6225754618644714),
 ('pleasant', 0.6126613616943359),
 ('good..', 0.6080995798110962),
 ('terrible', 0.6071532368659973),
 ('bad', 0.6029117107391357),
 ('fantastic', 0.5999585390090942),
 ('nice', 0.5962734818458557)]

In [618]:
print('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 30703


In [619]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [620]:
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, X_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, X_test))])
test_vecs_w2v = scale(test_vecs_w2v)






0it [00:00, ?it/s]/Users/synerzip/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  





544it [00:00, 5379.91it/s]




1107it [00:00, 5500.95it/s]




1645it [00:00, 5456.02it/s]




2182it [00:00, 5435.82it/s]




2745it [00:00, 5473.98it/s]




3280it [00:00, 5451.78it/s]




3803it [00:00, 5418.89it/s]




4353it [00:00, 5425.72it/s]




4875it [00:00, 5403.57it/s]




5394it [00:01, 5382.44it/s]




5934it [00:01, 5382.88it/s]




6516it [00:01, 5418.31it/s]




7056it [00:01, 5414.77it/s]




7600it [00:01, 5416.42it/s]




8141it [00:01, 5409.30it/s]




8698it [00:01, 5418.78it/s]




9242it [00:01, 5400.45it/s]




9776it [00:01, 5396.51it/s]




10310it [00:01, 5382.47it/s]




10852it [00:02, 5384.51it/s]




11396it [00:02, 5385.60it/s]




11932it [00:02, 5377.03it/s]




12490it [00:02, 5385.45it/s]




13029it [00:02, 5382.43it/s]

91560it [00:16, 5400.37it/s]




92110it [00:17, 5400.95it/s]




92660it [00:17, 5400.53it/s]




93205it [00:17, 5400.67it/s]




93750it [00:17, 5399.53it/s]




94292it [00:17, 5399.67it/s]




94833it [00:17, 5399.72it/s]




95382it [00:17, 5400.15it/s]




95937it [00:17, 5400.96it/s]




96483it [00:17, 5400.56it/s]




97025it [00:17, 5397.63it/s]




97555it [00:18, 5397.04it/s]




98083it [00:18, 5395.39it/s]




98606it [00:18, 5394.18it/s]




99128it [00:18, 5393.14it/s]




99649it [00:18, 5391.61it/s]




100168it [00:18, 5390.02it/s]




100697it [00:18, 5389.33it/s]




101222it [00:18, 5388.59it/s]




101760it [00:18, 5388.51it/s]




102289it [00:18, 5387.92it/s]




102819it [00:19, 5387.52it/s]




103368it [00:19, 5388.11it/s]




103923it [00:19, 5388.77it/s]




104464it [00:19, 5388.47it/s]




105002it [00:19, 5387.93it/s]




105550it [00:19, 5388.38it/s]




106091it [00:19, 5388.44it/s]




106646it [00:19, 5389.20it/s]




107190it [00:19, 5386.93it/s]


178986it [00:34, 5249.68it/s]




179530it [00:34, 5250.23it/s]




180073it [00:34, 5250.75it/s]




180611it [00:34, 5250.83it/s]




181146it [00:34, 5250.96it/s]




181685it [00:34, 5251.31it/s]




182220it [00:34, 5251.57it/s]




182785it [00:34, 5252.66it/s]




183328it [00:34, 5252.25it/s]




183862it [00:35, 5251.78it/s]




184394it [00:35, 5251.95it/s]




184922it [00:35, 5251.99it/s]




185457it [00:35, 5252.24it/s]




185987it [00:35, 5252.08it/s]




186544it [00:35, 5252.97it/s]




187080it [00:35, 5252.84it/s]




187612it [00:35, 5253.01it/s]




188144it [00:35, 5252.41it/s]




188669it [00:35, 5252.08it/s]




189192it [00:36, 5252.04it/s]




189713it [00:36, 5251.66it/s]




190248it [00:36, 5251.90it/s]




190776it [00:36, 5251.99it/s]




191318it [00:36, 5252.45it/s]




191848it [00:36, 5252.21it/s]




192393it [00:36, 5252.73it/s]




192949it [00:36, 5253.58it/s]




193488it [00:36, 5253.70it/s]




194025it [00:36, 5253.58it/s]




194572it [00:3

268215it [00:51, 5252.18it/s]




268749it [00:51, 5252.35it/s]




269281it [00:51, 5251.80it/s]




269810it [00:51, 5251.86it/s]




270337it [00:51, 5251.87it/s]




270862it [00:51, 5251.40it/s]




271381it [00:51, 5251.26it/s]




271903it [00:51, 5251.20it/s]




272435it [00:51, 5251.30it/s]




272958it [00:51, 5251.25it/s]




273481it [00:52, 5250.96it/s]




274024it [00:52, 5251.29it/s]




274557it [00:52, 5251.42it/s]




275095it [00:52, 5251.67it/s]




275626it [00:52, 5251.22it/s]




276167it [00:52, 5251.48it/s]




276699it [00:52, 5251.61it/s]




277258it [00:52, 5252.24it/s]




277796it [00:52, 5252.36it/s]




278333it [00:52, 5251.87it/s]




278859it [00:53, 5251.89it/s]




279385it [00:53, 5251.79it/s]




279909it [00:53, 5251.06it/s]




280424it [00:53, 5250.82it/s]




280944it [00:53, 5250.74it/s]




281471it [00:53, 5250.74it/s]




282002it [00:53, 5250.87it/s]




282543it [00:53, 5251.13it/s]




283071it [00:53, 5250.37it/s]




283593it [00:5

357026it [01:08, 5250.06it/s]




357557it [01:08, 5250.14it/s]




358090it [01:08, 5250.25it/s]




358618it [01:08, 5250.27it/s]




359145it [01:08, 5250.11it/s]




359680it [01:08, 5250.22it/s]




360207it [01:08, 5250.06it/s]




360730it [01:08, 5249.96it/s]




361259it [01:08, 5249.99it/s]




361792it [01:08, 5250.11it/s]




362357it [01:09, 5250.70it/s]




362895it [01:09, 5250.54it/s]




363426it [01:09, 5250.51it/s]




363975it [01:09, 5250.85it/s]




364510it [01:09, 5250.81it/s]




365042it [01:09, 5250.92it/s]




365574it [01:09, 5250.76it/s]




366111it [01:09, 5250.93it/s]




366641it [01:09, 5250.78it/s]




367200it [01:09, 5251.26it/s]




367736it [01:10, 5251.28it/s]




368269it [01:10, 5251.09it/s]




368816it [01:10, 5251.40it/s]




369353it [01:10, 5251.57it/s]




369887it [01:10, 5251.59it/s]




370447it [01:10, 5252.08it/s]




370995it [01:10, 5252.39it/s]




371538it [01:10, 5252.04it/s]




372072it [01:10, 5252.14it/s]




372616it [01:1

...

624885it [01:58, 5259.54it/s]




625416it [01:58, 5259.56it/s]




625947it [01:59, 5259.55it/s]




626487it [01:59, 5259.66it/s]




627034it [01:59, 5259.82it/s]




627571it [01:59, 5259.84it/s]




628105it [01:59, 5259.74it/s]




628643it [01:59, 5259.84it/s]




629174it [01:59, 5259.77it/s]




629701it [01:59, 5259.68it/s]




630258it [01:59, 5259.94it/s]




630792it [01:59, 5259.95it/s]




631329it [02:00, 5260.03it/s]




631864it [02:00, 5260.09it/s]




632398it [02:00, 5259.95it/s]




632928it [02:00, 5259.98it/s]




633456it [02:00, 5259.83it/s]




633989it [02:00, 5259.89it/s]




634518it [02:00, 5259.89it/s]




635049it [02:00, 5259.94it/s]




635577it [02:00, 5259.71it/s]




636116it [02:00, 5259.80it/s]




636655it [02:01, 5259.90it/s]




637186it [02:01, 5259.94it/s]




637715it [02:01, 5259.60it/s]




638244it [02:01, 5259.62it/s]




638765it [02:01, 5259.52it/s]




639284it [02:01, 5259.38it/s]




639828it [02:01, 5259.53it/s]




640353it [02:0

713391it [02:15, 5254.87it/s]




713930it [02:15, 5254.87it/s]




714468it [02:15, 5254.96it/s]




715004it [02:16, 5254.86it/s]




715534it [02:16, 5254.81it/s]




716067it [02:16, 5254.86it/s]




716595it [02:16, 5254.76it/s]




717126it [02:16, 5254.79it/s]




717674it [02:16, 5254.93it/s]




718206it [02:16, 5254.93it/s]




718736it [02:16, 5254.95it/s]




719265it [02:16, 5254.94it/s]




719803it [02:16, 5255.02it/s]




720340it [02:17, 5255.11it/s]




720882it [02:17, 5255.23it/s]




721433it [02:17, 5255.41it/s]




721973it [02:17, 5255.37it/s]




722515it [02:17, 5255.49it/s]




723064it [02:17, 5255.64it/s]




723604it [02:17, 5255.41it/s]




724131it [02:17, 5255.37it/s]




724680it [02:17, 5255.53it/s]




725225it [02:17, 5255.67it/s]




725761it [02:18, 5255.63it/s]




726292it [02:18, 5255.63it/s]




726821it [02:18, 5255.64it/s]




727350it [02:18, 5255.63it/s]




727911it [02:18, 5255.88it/s]




728449it [02:18, 5255.90it/s]




728984it [02:1

802330it [02:32, 5255.05it/s]




802873it [02:32, 5255.15it/s]




803416it [02:32, 5255.25it/s]




803958it [02:32, 5255.36it/s]




804498it [02:33, 5255.38it/s]




805060it [02:33, 5255.61it/s]




805622it [02:33, 5255.84it/s]




806220it [02:33, 5256.30it/s]




806784it [02:33, 5256.55it/s]




807347it [02:33, 5256.68it/s]




807905it [02:33, 5256.73it/s]




808464it [02:33, 5256.95it/s]




809017it [02:33, 5256.93it/s]




809561it [02:33, 5256.93it/s]




810114it [02:34, 5257.11it/s]




810674it [02:34, 5257.31it/s]




811224it [02:34, 5257.48it/s]




811773it [02:34, 5257.48it/s]




812340it [02:34, 5257.76it/s]




812890it [02:34, 5257.87it/s]




813465it [02:34, 5258.18it/s]




814021it [02:34, 5258.29it/s]




814593it [02:34, 5258.57it/s]




815150it [02:35, 5258.61it/s]




815732it [02:35, 5258.97it/s]




816304it [02:35, 5259.25it/s]




816867it [02:35, 5259.35it/s]




817423it [02:35, 5259.42it/s]




817973it [02:35, 5259.39it/s]




818522it [02:3

895258it [02:49, 5277.50it/s]




895810it [02:49, 5277.48it/s]




896353it [02:49, 5277.44it/s]




896908it [02:49, 5277.59it/s]




897450it [02:50, 5277.62it/s]




897989it [02:50, 5277.45it/s]




898517it [02:50, 5277.39it/s]




899059it [02:50, 5277.47it/s]




899598it [02:50, 5277.53it/s]




900131it [02:50, 5277.54it/s]




900663it [02:50, 5277.52it/s]




901194it [02:50, 5277.54it/s]




901727it [02:50, 5277.56it/s]




902258it [02:50, 5277.51it/s]




902787it [02:51, 5277.51it/s]




903331it [02:51, 5277.60it/s]




903887it [02:51, 5277.76it/s]




904454it [02:51, 5277.99it/s]




905002it [02:51, 5277.97it/s]




905561it [02:51, 5278.16it/s]




906118it [02:51, 5278.31it/s]




906667it [02:51, 5278.31it/s]




907220it [02:51, 5278.45it/s]




907775it [02:51, 5278.61it/s]




908324it [02:52, 5278.65it/s]




908893it [02:52, 5278.89it/s]




909445it [02:52, 5278.97it/s]




910057it [02:52, 5279.46it/s]




910627it [02:52, 5279.70it/s]




911195it [02:5

987567it [03:06, 5292.01it/s]




988117it [03:06, 5292.01it/s]




988661it [03:06, 5291.81it/s]




989191it [03:06, 5291.73it/s]




989772it [03:07, 5292.00it/s]




990319it [03:07, 5292.08it/s]




990870it [03:07, 5292.20it/s]




991415it [03:07, 5292.26it/s]




991958it [03:07, 5292.25it/s]




992497it [03:07, 5291.84it/s]




993013it [03:07, 5291.71it/s]




993574it [03:07, 5291.89it/s]




994102it [03:07, 5291.86it/s]




994629it [03:07, 5291.71it/s]




995170it [03:08, 5291.77it/s]




995717it [03:08, 5291.86it/s]




996264it [03:08, 5291.95it/s]




996826it [03:08, 5292.12it/s]




997381it [03:08, 5292.25it/s]




997928it [03:08, 5292.22it/s]




998468it [03:08, 5292.20it/s]




999009it [03:08, 5292.26it/s]




999553it [03:08, 5292.33it/s]




1000000it [03:08, 5292.44it/s]




0it [00:00, ?it/s]




243it [00:00, 2402.75it/s]




466it [00:00, 2313.74it/s]




696it [00:00, 2305.22it/s]




905it [00:00, 2250.30it/s]




1124it [00:00, 2239.16it/s]




1366

In [622]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = n_dim))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.fit(train_vecs_w2v, y_train, epochs=9, batch_size=32, verbose=2)

Epoch 1/9
 - 46s - loss: 0.3546 - acc: 0.8519
Epoch 2/9
 - 45s - loss: 0.3415 - acc: 0.8574
Epoch 3/9
 - 45s - loss: 0.3381 - acc: 0.8590
Epoch 4/9
 - 46s - loss: 0.3363 - acc: 0.8598
Epoch 5/9
 - 45s - loss: 0.3349 - acc: 0.8604
Epoch 6/9
 - 45s - loss: 0.3341 - acc: 0.8610
Epoch 7/9
 - 45s - loss: 0.3333 - acc: 0.8612
Epoch 8/9
 - 45s - loss: 0.3327 - acc: 0.8614
Epoch 9/9
 - 45s - loss: 0.3323 - acc: 0.8616


In [623]:
y_pred = model.predict(test_vecs_w2v)

In [628]:
y_pred

array([[ 0.0950716 ],
       [ 0.94019449],
       [ 0.22971125],
       ..., 
       [ 0.00751881],
       [ 0.29025608],
       [ 0.07534358]], dtype=float32)

In [629]:
df_pred = pd.DataFrame()
df_pred['tweet'] = tweets_full_text
df_pred['pred'] = y_pred
df_pred.to_csv('data/result.csv', index=False)

In [630]:
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(train_vecs_w2v, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [633]:
y_pred = model.predict(test_vecs_w2v)

In [634]:
accuracies = cross_val_score(estimator = model, X = train_vecs_w2v, y = y_train, cv = 10)

KeyboardInterrupt: 

In [ ]:
accuracies.mean()

In [ ]:
y_pred

In [ ]:
df_pred = pd.DataFrame()
df_pred['tweet'] = tweets_full_text
df_pred['pred'] = y_pred
df_pred.to_csv('data/gov-sent-random-forest-result.csv', index=False)